# Como converter objeto Anndata para Seurat (reproduzir essa parte no Python) --> Exemplo Sanbomics
# Existem algumas formas de conversão direta entre um objeto Anndata para Seurat preservando toda análise, mas surgiram alguns erros e problemas no output final
# Apesar de mais trabalhoso, recomendo converter os arquivos brutos e realizar análise do começo ao fim
# Criamos um diretório para salvar os arquivos (barcodes, features e matrix)

!mkdir matrix_files

# Salvando barcodes, features e matrix

with open('matrix_files/barcodes.tsv', 'w') as f:
  for item in adata.obs_names:
  f.write(item + '\n')

with open('matrix_files/features.tsv', 'w') as f:
  for item in ['\t'.join([x,x,'Gene Expression']) for x in adata.var_names]:
  f.write(item + '\n')

io.mmwrite('matrix_files/matrix', adata.X.T)

# Salvando o metadata

adata.obs.to_csv('matrix_files/metadata.csv')




# Aqui já é diretamente no R

# Pacotes


In [ ]:
library(BiocManager)
library(dplyr)
library(pheatmap)
library(ggplot2)
library(Seurat)
library(harmony)
library(infercnv)
library(CellChat)
library(monocle)
library(tibble)
library(SingleCellExperiment)
library(purrr)
library(magrittr)
library(edgeR)
library(Matrix)
library(DESeq2)
library(VennDiagram)
library(muscat)
library(SingleR)
library(enrichR)
library(msigdbr)
library(Matrix.utils)
library(apeglm)
library(UpSetR)
library(RColorBrewer)
library(scater)
library(tidyr)
library(SeuratWrappers)
library(remotes)
library(Rmagic)
library(UCell)
library(ggpubr)
library(Scillus)
library(viridis)
library(WGCNA)
library(hdWGCNA)
library(patchwork)
library(igraph)
library(cowplot)
library(tidyverse)
library(gridGraphics)
library(gridExtra)
library(ggplotify)
library(ComplexHeatmap)
library(iDEA)
library(clusterProfiler)
library(cola)
library(corrplot)
library(plot3D)
library(plotly)
library(dyno)
library(Nebulosa)
library(EnhancedVolcano)
library(InterCellar)
library(miloR)
library(symphony)
library(SeuratDisk)
library(BisqueRNA)



# Carregando o metadata. Aqui já temos as doublets identificadas e vamos removê-las


In [ ]:
metadata <- read.csv("D:/Scanpy/Workflow_A/metadata_workflow_peng.csv")



# Por configurações diferentes entre Python e R, os barcodes estão na primeira coluna e não em rownames
# Arrumamos isso e deletamos a coluna


In [ ]:
rownames(metadata) <- metadata$X
metadata$X <- NULL
View(metadata)



# Importando os barcodes, features e matrix


In [ ]:
counts <- readMM("D:/Scanpy/peng_to_seurat/matrix.mtx")
genes <- read_tsv("D:/Scanpy/peng_to_seurat/features.tsv", col_names = FALSE)
cell_ids <- read_tsv("D:/Scanpy/peng_to_seurat/barcodes.tsv", col_names = FALSE)



# Configurando a matriz para que nas linhas (rownames) tenhamos os genes e nas colunas (colnames) os barcodes


In [ ]:
rownames(counts) <- genes$X1
colnames(counts) <- cell_ids$X1



# Criando o objeto Seurat


In [ ]:
sobj <- CreateSeuratObject(counts = counts, meta.data = metadata)
sobj



# Vamos iniciar removendo as doublets


In [ ]:
sobj <- SetIdent(sobj, value = sobj$doublet_pred)
sobj <- subset(sobj, idents = "singlet")
sobj



# Vamos plotar os gráficos de QC (quality-control)


In [ ]:
VlnPlot(sobj, features = c("n_genes_by_counts", "total_counts", "pct_counts_ribo", "pct_counts_hb"), ncol = 4)
FeatureScatter(sobj, feature1 = "total_counts", feature2 = "n_genes_by_counts")

metadata <- sobj@meta.data

metadata %>% 
  ggplot(aes(x=Patient, fill=Patient)) + 
  geom_bar() +
  theme_classic() +
  theme(axis.text.x = element_text(angle = 90, vjust = 1, hjust=1)) +
  theme(plot.title = element_text(hjust=0.5, face="bold")) +
  ggtitle("Número de células por paciente")

metadata %>% 
  ggplot(aes(x=Patient, y=log10(n_genes_by_counts), fill=Patient)) + 
  geom_boxplot() + 
  theme_classic() +
  theme(axis.text.x = element_text(angle = 90, vjust = 1, hjust=1)) +
  theme(plot.title = element_text(hjust=0.5, face="bold")) +
  ggtitle("Genes encontrados por paciente")



# Normalização com LogNormalize, Cell-cycle score e identificação dos 3000 HVGs


In [ ]:
sobj <- NormalizeData(object = sobj, normalization.method = "LogNormalize", scale.factor = 10000)
s.genes <- cc.genes$s.genes
g2m.genes <- cc.genes$g2m.genes
sobj <- CellCycleScoring(sobj, g2m.features = g2m.genes, 
                                 s.features = s.genes)
sobj$CC.Difference <- sobj$S.Score - sobj$G2M.Score
sobj <- FindVariableFeatures(sobj, selection.method = "vst", nfeatures = 3000, verbose = FALSE)



# Scale data e PCA. Indicamos no argumento features que vamos realizar o scaling somente nos HVGs


In [ ]:
sobj <- ScaleData(sobj, features = VariableFeatures(sobj))
sobj <- RunPCA(sobj, npcs = 50, verbose = TRUE)



# Identificando o melhor número de PCs para incluir nos passos downstream


In [ ]:
ElbowPlot(sobj, ndims = 50)



# Olhando o Elbowplot acima percebemos que um bom número de PCs está entre 30-35. Dificilmente teremos grandes diferenças na clusterização se optarmos por usar 35 < nPCs. Podemos aplicar uma fórmula para verificar quanto de variância está sendo capturada com 30-35 PCs. Com 30 PCs, já capturamos 90% da variância.


In [ ]:
stdev <- sobj@reductions$pca@stdev
var <- stdev^2
sum(var[1:30])/ sum(var)
sum(var[1:35])/ sum(var)


# Integração e remoção do efeito de batch com Harmony


In [ ]:
sobj <- RunHarmony(sobj, group.by.vars = "Patient")




# FindNeighbors, UMAP e visualização


In [ ]:
sobj <- FindNeighbors(object = sobj, dims = 1:30, reduction = "harmony")
sobj <- RunUMAP(object = sobj, dims = 1:30, reduction = "harmony")

p1<-DimPlot(object = sobj, reduction = "umap", group.by = "Condition")
p2<-DimPlot(object = sobj, reduction = "umap", group.by = "Patient")+NoLegend()
p1+p2



# Clusterização


In [ ]:
for (i in c(0.3, 0.5, 0.7)) {
  sobj <- FindClusters(sobj, resolution = i, reduction = "harmony")
  print(DimPlot(sobj, reduction = "umap") + labs(title = paste0("resolution: ", i)))
}


# FeaturePlot, DotPlot e Heatmap para anotação dos tipos celulares


In [ ]:
features <- c("EPCAM", "CTRB1", "FXYD2", "FXYD3", "DCN", "COL1A1",
                               "PLVAP", "ACTA2", "PTPRC", "CD79A", "MZB1", "CD3D",
                               "NKG7", "CD68", "CD14", "AIF1")
FeaturePlot(sobj, features = features, cols = c("lightgrey", "red"), raster = TRUE, ncol = 4)
DotPlot(object = sobj, features = features, idents = sobj$RNA_snn_res.0.3, cols = c("lightgrey", "red"))+RotatedAxis()
FeaturePlot(sobj, features = "INS", cols = c("lightgrey", "red"))


# Expressão diferencial entre os clusters para identificação de marcadores


In [ ]:
markers <- FindAllMarkers(object = sobj, logfc.threshold = 0.5, test.use = "roc")
head(markers)



# Assinaturas gênicas com UCell


In [ ]:
tnk_genes <- c("CD3D", "IL7R", "CD3E", "CD8A", "NKG7")
fibro_genes <- c("COL1A1", "DCN", "CTHRC1", "MMP11", "C7")
duct_genes <- c("CFTR", "AMBP", "FXYD2", "EPCAM")
stellate_genes <- c("ACTA2", "RGS5", "PDGFRB", "MYH11", "COL1A1")
myeloid_genes <- c("CD14", "CD68", "HLA-DRA", "AIF1", "APOE")
b_genes <- c("MS4A1", "CD19", "CD79A", "CD27", "CD79B")
plasma_genes <- c("MZB1", "IGLL5", "JCHAIN", "DERL3")
endo_genes <- c("PLVAP", "VWF", "CDH5", "CLDN5")
endoc_genes <- c("INS", "SST", "TTR")
malig_genes <- c("MUC1", "KRT19", "FXYD3", "SOX9", "TFF1", "TFF2", "TFF3")
acinar_genes <- c("CPB1", "SPINK1", "PRSS1", "CTRB1", "AMY2A", "TRY4")

cell_markers <- list()
cell_markers$Tnk <- c(tnk_genes)
cell_markers$Fibro <- c(fibro_genes)
cell_markers$Duct <- c(duct_genes)
cell_markers$Stellate <- c(stellate_genes)
cell_markers$Myeloid <- c(myeloid_genes)
cell_markers$Bcells <- c(b_genes)
cell_markers$Plasma <- c(plasma_genes)
cell_markers$Endothelial <- c(endo_genes)
cell_markers$Endocrine <- c(endoc_genes)
cell_markers$Malignant <- c(malig_genes)
cell_markers$Acinar <- c(acinar_genes)

sobj <- AddModuleScore_UCell(sobj, features = cell_markers)
signature.names <- paste0(names(cell_markers), "_UCell")


# Visualização dos scores


In [ ]:
DotPlot(sobj, features = signature.names, cols = c("lightgrey", "red"))+RotatedAxis()




# Salvando o dataframe com os marcadores


In [ ]:
write.table(markers,file='D:Scanpy/Workflow_A/roc_markers_seurat.csv',
  quote=F,row.names=F,col.names=F
)



# Anotação dos clusters


In [ ]:
sobj <- SetIdent(sobj, value = sobj$RNA_snn_res.0.3)
cell_ids <- c("Ductal cells 2", "Ductal cells 1", "Endothelial cells", "Fibroblasts",
              "Stellate cells", "Myeloid cells", "T-NK cells", "B cells", "Acinar cells",
              "Mixed cycling cells", "Endothelial cells", "Endocrine cells", "Plasma cells",
              "Endothelial cells", "Myeloid cells", "Ductal cells 1", "Stellate cells")
names(cell_ids) <- levels(sobj)
sobj <- RenameIdents(sobj, cell_ids)
sobj[["cell_type"]] <- Idents(object = sobj)
DimPlot(sobj, reduction = "umap")


# Salvando objeto Seurat


In [ ]:
saveRDS(object = sobj, file = 'D:/Scanpy/Workflow_A/seurat_peng.rds')




# Composição celular


In [ ]:
composition1 <- table(Idents(sobj), sobj$Condition)
composition1 <- as.data.frame(composition1)
composition1$Var1 <- as.character(composition1$Var1)
composition1$Var2 <- as.character(composition1$Var2)
composition1 = composition1 %>% rename(Cell_Type = "Var1", Condition = "Var2") %>%
  group_by(Condition) %>%
  mutate(Percent = Freq / sum(Freq)*100)

composition2 <- table(Idents(sobj), sobj$Patient)
composition2 <- as.data.frame(composition2)
composition2$Var1 <- as.character(composition2$Var1)
composition2$Var2 <- as.character(composition2$Var2)
composition2 = composition2 %>% rename(Cell_Type = "Var1", Patient = "Var2") %>%
  group_by(Patient) %>%
  mutate(Percent = Freq / sum(Freq)*100)
composition2$Patient <- factor(composition2$Patient,
                                      levels = c("N1", "N2", "N3", "N4", "N5", "N6", "N7",
                                                 "N8", "N9", "N10", "N11", "T1", "T2", "T3",
                                                 "T4", "T5", "T6", "T7", "T8", "T9", "T10",
                                                 "T11", "T12", "T13", "T14", "T15", "T16", "T17",
                                                 "T18", "T19", "T20", "T21", "T22", "T23", "T24"))

p1 <- ggplot(composition1, aes(x = Condition, y = Percent, fill = Cell_Type)) +
  geom_bar(stat = "identity", color = "black") +
  theme_bw()+ggtitle("Cell type proportion per condition")

p2<-  ggplot(composition2, aes(x = Patient, y = Percent, fill = Cell_Type)) +
  geom_bar(stat = "identity", color = "black") +
  theme_bw()+theme(axis.text.x = element_text(angle = 90, size = 8, hjust = 1))+ggtitle("Cell type proportion per patient")

p1+p2
